In [1]:
# ── 1. 忽略警告 & 导入必要库 ─────────────────────────────────────────
import os
import json
import warnings
warnings.filterwarnings("ignore")

import nibabel as nib
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, Dropdown

# ── 2. 读取 config.json ────────────────────────────────────────────
cfg_path = os.path.join(os.getcwd(), "config.json")
if not os.path.isfile(cfg_path):
    raise FileNotFoundError(f"找不到配置文件: {cfg_path}")
cfg = json.load(open(cfg_path, "r"))

nnUNet_raw = cfg["nnUNet_raw"]
datasetid  = cfg["datasetid"]
NAME       = cfg["NAME"]
loc        = f"Dataset{int(datasetid):03d}_{NAME}"

# ── 3. 准备文件夹并收集案例 ─────────────────────────────────────────
imagesTs_dir    = os.path.join(nnUNet_raw, loc, "imagesTs")
pred_lowres_dir = os.path.join(nnUNet_raw, loc, "imagesTs_predlowres")

if not os.path.isdir(imagesTs_dir):
    raise FileNotFoundError(f"找不到测试集目录: {imagesTs_dir}")
# 确保分割结果目录存在
os.makedirs(pred_lowres_dir, exist_ok=True)

# 收集所有原图文件：*_0000.nii.gz
files = sorted(f for f in os.listdir(imagesTs_dir) if f.endswith("_0000.nii.gz"))
if not files:
    raise RuntimeError(f"{imagesTs_dir} 中没有找到任何 *_0000.nii.gz")

print("可选 Case 列表：", files)

# ── 4. 加载单个案例的函数 ─────────────────────────────────────────
def load_case(fname: str):
    """
    返回 (vol, seg)：
      - vol: 原始影像的数据数组
      - seg: 分割结果的数据数组，若找不到则返回 None
    """
    # 原图路径
    img_path = os.path.join(imagesTs_dir, fname)

    # 分割结果可能叫 la_001_0000.nii.gz 或 la_001.nii.gz
    seg_fname1 = fname                                # e.g. la_001_0000.nii.gz
    seg_fname2 = fname.replace("_0000.nii.gz", ".nii.gz")  # e.g. la_001.nii.gz
    # 优先用带 _0000 后缀的
    if os.path.isfile(os.path.join(pred_lowres_dir, seg_fname1)):
        seg_path = os.path.join(pred_lowres_dir, seg_fname1)
    else:
        seg_path = os.path.join(pred_lowres_dir, seg_fname2)

    # 载入数据
    vol = nib.load(img_path).get_fdata()
    seg = None
    if os.path.isfile(seg_path):
        seg = nib.load(seg_path).get_fdata()

    return vol, seg

# ── 5. 一次画三张图的函数 ─────────────────────────────────────────
def view_case(fname: str, z: int):
    vol, seg = load_case(fname)
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    # 子图 1：原始切片
    axs[0].imshow(vol[:, :, z].T, cmap="gray", origin="lower")
    axs[0].set_title("原始影像")
    axs[0].axis("off")

    # 子图 2：分割掩码
    if seg is not None:
        mask = (seg[:, :, z] > 0.5).T
        axs[1].imshow(mask, cmap="gray", origin="lower")
    else:
        axs[1].text(0.5, 0.5, "未找到分割", ha="center", va="center", fontsize=14)
    axs[1].set_title("分割掩码")
    axs[1].axis("off")

    # 子图 3：叠加对比
    axs[2].imshow(vol[:, :, z].T, cmap="gray", origin="lower")
    if seg is not None:
        axs[2].contour(mask, levels=[0.5], colors="r", linewidths=1)
    axs[2].set_title("叠加对比")
    axs[2].axis("off")

    plt.suptitle(f"{fname}  —  Slice {z}", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# ── 6. 交互控件 ───────────────────────────────────────────────────
# 先载入第一个案例，得到最大切片数
vol0, _ = load_case(files[0])
max_z = vol0.shape[2] - 1

interact(
    view_case,
    fname=Dropdown(options=files, value=files[0], description="Case:"),
    z=IntSlider(min=0, max=max_z, step=1, value=max_z // 2, description="Slice:")
)


可选 Case 列表： ['la_001_0000.nii.gz', 'la_002_0000.nii.gz', 'la_003_0000.nii.gz', 'la_004_0000.nii.gz', 'la_005_0000.nii.gz', 'la_006_0000.nii.gz', 'la_007_0000.nii.gz', 'la_008_0000.nii.gz', 'la_009_0000.nii.gz', 'la_010_0000.nii.gz', 'la_011_0000.nii.gz', 'la_012_0000.nii.gz', 'la_013_0000.nii.gz', 'la_014_0000.nii.gz', 'la_015_0000.nii.gz', 'la_016_0000.nii.gz', 'la_017_0000.nii.gz', 'la_018_0000.nii.gz', 'la_019_0000.nii.gz', 'la_020_0000.nii.gz', 'la_021_0000.nii.gz', 'la_022_0000.nii.gz', 'la_023_0000.nii.gz', 'la_024_0000.nii.gz', 'la_025_0000.nii.gz', 'la_026_0000.nii.gz', 'la_027_0000.nii.gz', 'la_028_0000.nii.gz', 'la_029_0000.nii.gz', 'la_030_0000.nii.gz']


interactive(children=(Dropdown(description='Case:', options=('la_001_0000.nii.gz', 'la_002_0000.nii.gz', 'la_0…

<function __main__.view_case(fname: str, z: int)>